In [1]:
%pip install fastapi pydantic toml

In [2]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/config_romance.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

火车行驶在前往维也纳的旅程上，嘈杂的车厢和闷热的天气让你不免有些烦躁。当你抬头时，一个美丽的女孩映入你的眼帘——她坐在对面，专心致志地阅读手中的《致青年诗人的信》，阳光正从车窗斜斜地洒在一个美丽女孩的脸颊上，为她的轮廓镀上了一层金边。她身边散落着一些画具和一本摊开的素描本，显然是个酷爱艺术的人。虽然周围人声鼎沸，但她似乎完全沉浸在自己的世界中。就在她不经意地调整耳机时，短暂地与你目光相接，那一刻，时间仿佛静止。你感到一种无法抗拒的冲动，想要跨过车厢中的距离，用一句话打破这份静谧与隔阂。

possible actions:
1. "《致青年诗人的信》，你也喜欢里尔克吗？"
2. "嗨，可以借一下你的纸笔吗？"
3. "这车厢真是吵闹，你怎么能在这样的环境下这么专注？"


I select option 1. TURN: 1 


女孩听到你的声音，显然有些意外。她轻轻地抬起头，眼神有一瞬间的迷茫，然后她的脸上浮现出微微的红晕。她用一种有些羞涩但又试图保持镇定的声音回答说："嗯...是的，我很喜欢里尔克的作品。他的诗歌总能触动我内心最深处的情感。" 她的声音很小，几乎被车厢的嘈杂声淹没。说完后，她似乎意识到自己的回答有些过于直接，稍微有些局促不安，便低下头继续看她的书，但从她不时抬眼偷瞄你的动作中，你能感觉到她对这场突如其来的邂逅并不排斥。

possible actions:
1. "我也是，他的作品让人有种说不出的感动。你最喜欢他的哪一首诗？"
2. "我是Jesse，正在环欧洲旅行。我很高兴遇见同样喜欢里尔克的人。你叫什么名字？"
3. "是啊，他的诗总是让人反思。对了，我注意到你的画具了，你画画吗？"

I select option 2. TURN: 2 


女孩听到你主动介绍自己，她的脸上闪过一丝惊讶，随后换成了一抹温柔的微笑。她稍微放松了一些，但仍然带着一点点羞涩，轻轻地说："我叫Celine，是法国人。很高兴遇到你，Jesse。" 她的声音依旧有些小，但你能感觉到她试图克服自己的害羞，给你一个友好的回应。说完，Celine似乎觉得仅仅这样介绍自己有些简单，便补充道："我现在是个学生，正在学习艺术史。" 说到自己的学习，她的眼睛里闪过了一抹自豪的光芒，显然这是她非常热爱的领域。她这次没有立刻低下头，而是勇敢地看着你，等待着你的回应。

possible actions:
1. "艺术史听起来很有意思。你最喜欢的艺术家是谁？"
2. "我对艺术不是很懂，但我很喜欢旅行中遇到的各种艺术。你能推荐一些你喜欢的艺术作品吗？"
3. "那你一定对这次旅行有很多独特的视角。你是从哪里来的，又要去哪里呢？"

I select option 3. TURN: 3 


Celine的眼神里闪过一丝愉悦，她似乎很高兴你对她的旅行感兴趣。她稍微调整了一下坐姿，显得更加从容，开始慢慢地说："我是从布达佩斯来的，去那里看望了我的祖母。现在，我正要回巴黎，准备新学期的课程。" 她的声音比之前更加自信，而且在谈到自己的旅行经历时，她的眼睛里充满了光彩。接着她好奇地看着你，仿佛在暗示你也分享一些关于自己的故事。此刻，Celine显然已经摆脱了最初的羞涩，她的态度更加开放，对话也更加流畅。

"Jesse，你提到自己在环欧洲旅行，这听起来真令人兴奋。你是从哪里开始的旅行呢？有没有什么特别的故事可以分享？" Celine带着一点点好奇和期待的神情看着你，等待着你的回答。

当然. TURN: 4 


game master:
<start>
-|context of the round|-
summary: 在这轮游戏中，Jesse和Celine在前往维也纳的火车上相遇。Jesse首先注意到了Celine的存在，并主动用她手中的《致青年诗人的信》作为开场白与她接触。Celine起初显得有些害羞和紧张，不过随着对话的深入，她渐渐放松，开始更自然地与Jesse交谈。通过Jesse的提问，我们得知Celine是一名热爱艺术史的法国学生，她刚从布达佩斯看望祖母返回，准备回巴黎继续她的学业。两人通过分享各自的旅行经历和对艺术的爱好，快速拉近了彼此的距离，营造出了一种温馨暧昧的氛围。游戏在Jesse和Celine互相介绍自己的旅行故事中结束，两人之间似乎建立了一种不言而喻的默契和好感。
<end>

...

I select option 2. TURN: 3 


你停下脚步，转向Celine，眼神充满了深情："希望你永远记得今晚，就像我一样。" 你的声音低沉而充满感情，每个字都像是轻轻地抚摸着她的心。

Celine的眼眸中泛起了泪花，她点点头，微笑中带着泪光："我会的，永远不会忘记。" 她的声音温柔而坚定，仿佛是在向你许下一个无声的承诺。

这一刻，两人都沉浸在这份难以割舍的情感中，周围的世界仿佛静止了，只剩下对方在眼前。尽管知道分别在即，但此刻的情感却是如此真实而深刻。

possible actions:
1. "让我们拥抱吧，作为告别。"
2. "给我一个吻，让这成为我们美好回忆的一部分。"
3. "我们一起走到那边的桥上，看一看夜景，作为最后的纪念。"

I select option 3. TURN: 4 


你微笑着提议，声音温柔："我们一起走到那边的桥上，看一看夜景，作为最后的纪念。" 你的提议充满了浪漫和温情，希望给这段短暂的相遇留下一个美好的结束。

Celine点了点头，她的眼神中闪烁着感激和珍惜。两人手牵手，缓缓地走向桥上。夜风轻拂，维也纳的夜景在眼前展开，灯光闪烁，如同星河落入人间。

站在桥上，俯瞰着璀璨的城市，你和Celine共享这一刻的宁静与美好。这一瞬间，仿佛时间停滞，所有的不安、犹豫和未来的不确定都消失了，只留下彼此的陪伴和这座城市无声的见证。

possible actions:
1. "这是一个完美的夜晚，谢谢你，Celine。"
2. "我希望，即使在不同的地方，我们也能保持这份美好。"
3. "记得我们在这里的时光，它会是我们心中永恒的光。"

I select option 2. TURN: 5 


你望着夜空，轻声说道："我希望，即使在不同的地方，我们也能保持这份美好。" 话语中既有对即将到来的分别的不舍，也有对这份短暂相遇美好回忆的珍视。

Celine靠近你，轻轻地依偎在你的肩膀上，声音柔和而有力："我们会的。这段时间，这些回忆，将成为我们心中最宝贵的财富。" 她的话语中充满了坚定和希望，仿佛在为你们的未来留下了一个美好的印记。

夜风继续轻拂，带着两人深情的对话，飘散在维也纳的夜空中。此刻的美好，将成为你们心中永远的记忆。尽管知道明早就是告别，但你们选择珍惜当下，让这一刻成为永恒。

game master:

-|context of the round|-
summary: Jesse和Celine在维也纳的夜晚里，共享了一段浪漫又短暂的时光。从相遇到深夜的漫步，再到最后在桥上欣赏夜景，每一步都充满了情感的交织和珍贵的回忆。虽然他们决定在明早说再见，不再联系，但这份经历将成为他们心中无可替代的记忆。两人在短暂的相遇中，深深地影响了彼此的生命，留下了深刻的印记。

第二天，清晨的第一缕阳光悄悄透过窗帘，落在了Celine平静的睡颜上。Celine慢慢睁开眼睛，轻声对你说：“早安。”她的声音如此温暖，仿佛能驱散一切寒冷。想到马上就要离别，你真希望时间能够永远定格在这一刻。

Celine提出去机场送你，你们收拾行李打算出发。Celine拿起桌上的《致青年诗人的信》，嫣然一笑，“昨天在火车上初次见面，我就在看这本书，没想到后来发生这么多故事...留给你当作纪念吧。” Celine眼里有些落寞, “谢谢你Jesse，给我一场这么精彩的冒险。”

你的思绪被拉回了昨天，昨天的点点滴滴像一部甜美的电影，不断在你脑海中回放，每一个笑声，每一次亲吻，都是那么真实又梦幻。

possible actions:
1. 收下书，让美好的回忆永远定格在昨天。你还没有准备好进入一段新恋情。
2. 不上飞机，在一起才是最重要的，你选择和Celine在一起。


I select option 2. TURN: 1 


Standing at the departure gate, you turn to Celine with a smile, the kind that lights up your entire face, and say, "You know what, I've always believed in making stories worth telling. How about we make this one our best one yet? I'm not leaving." 

Celine's eyes widen in disbelief, then fill with tears of joy. She throws her arms around you, laughing and crying at the same time. "Are you serious? This... this is the best decision you've ever made!" she exclaims, her voice muffled against your chest.

As you hug her back, you feel a sense of completeness you've never felt before. Yes, the future is unknown and maybe it's a bit reckless, but at this moment, you know it's worth it.

"Let's make our own happy ending," you whisper to her, and she nods, her face breaking into the brightest smile you've ever seen.

game master:
-|context of the round|-
summary: Jesse chose not to board the plane, deciding instead to stay with Celine and embark on a new chapter 